In [11]:
import sys
sys.path.append("/Users/karinabalagazova/Desktop/cvut/5.semestr/scientificProject/notebooks/..")
import os.path
# from functions.experiment import experiment_iteration
from functions.rnaseq_data_generator import rna_seq_generator, NormalDistributionParameters
from utils.enums import Distribution
%load_ext autoreload
%autoreload 2
import numpy as np

from functions.saving_data import get_empty_auc_dataset
from multiprocessing import Pool

In [12]:
# Prepare to store data into array
STA_range = [20, 50, 100, 200, 500, 1000, 2000, 5000]
data = get_empty_auc_dataset(STA_range, 2)

In [13]:
# Save data
# np.savez('auc_results_origin.npz', auc_data=data)
saved_origin = np.load('auc_results_origin.npz', allow_pickle=True)
saved_origin = saved_origin['auc_data'].item()

# np.savez('auc_results_expected.npz', auc_data=data)
saved_expected = np.load('auc_results_expected.npz', allow_pickle=True)
saved_expected = saved_expected['auc_data'].item()

# np.savez('auc_results_filtered.npz', auc_data=data)
saved_filtered = np.load('auc_results_filtered.npz', allow_pickle=True)
saved_filtered = saved_filtered['auc_data'].item()

In [14]:
import statsmodels.api as sm

def work_bitch(df):
#     if distribution == "Poisson":
    mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.Poisson(), data=df).fit()
#     elif distribution == "Negative binomial":
#     mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.NegativeBinomial(), data=df).fit()
    return np.array([mp.params.x1, mp.params.x2, mp.params.x3])
#     else:
#         print("Unknown distribution")
#         return

def compensation(counts, IS, t, n_CR, distribution, pool, filename):
    """
    IS compensation.

    :param counts: Counts to be compensated.
    :param IS: Array of immunosuppression intake (0 or 1).
    :param t: Number of transcripts.
    :param distribution: "Poisson" or "Negative binomial".
    :param n_CR: Number of samples in CR class. First n_CR elements of counts must be of CR class.
    """
    # leave only STA
    STA_patients = counts.columns.str.startswith('STA')
    STA_counts = counts.loc[:, STA_patients].to_numpy()
    STA_IS = IS.loc[STA_patients]

    # try to learn the beta1 parameters from STA class
    # TODO: save learned_beta and use it to speed up code
    learned_beta = np.zeros((3, t))
    values = list()
    for ind in range(t):
        values.append(pd.DataFrame({"x1": STA_IS['IS1'].values.tolist(),
                           "x2": STA_IS['IS2'].values.tolist(),
                           "x3": STA_IS['IS3'].values.tolist(),
                           "y": STA_counts[ind]}))

#     with Pool() as pool:
    vs = pool.map(work_bitch, values)

    for ind in range(t):
        learned_beta[:, ind] = vs[ind]
#     return
#         df = pd.DataFrame({"x1": STA_IS['IS1'].values.tolist(),
#                            "x2": STA_IS['IS2'].values.tolist(),
#                            "x3": STA_IS['IS3'].values.tolist(),
#                            "y": STA_counts[ind]})
#         if distribution == "Poisson":
#             mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.Poisson(), data=df).fit()
#         elif distribution == "Negative binomial":
#             mp = sm.formula.glm("y ~ x1 + x2 + x3", family=sm.families.NegativeBinomial(), data=df).fit()
#         else:
#             print("Unknown distribution")
#             return
#         learned_beta[:, ind] = np.array([mp.params.x1, mp.params.x2, mp.params.x3])


    np.savez('{filename}_learnedbeta.npz'.format(filename=filename), beta=learned_beta)
    
    # leave only CR and OT classes for classification
    OT_CR_counts = counts.loc[:, ~STA_patients].to_numpy(dtype=float)
    OT_CR_IS = IS.loc[~STA_patients].values

    # filter out IS from CR
    OT_CR_counts[:, :n_CR] = OT_CR_counts[:, :n_CR] / np.exp(np.dot(OT_CR_IS[:n_CR], learned_beta).transpose())
    OT_CR_counts = OT_CR_counts.transpose()
    return OT_CR_counts, OT_CR_IS, learned_beta

In [16]:
def experiment(generated_counts, generated_expected_counts, generated_IS, n_CR: int, distribution, pool, filename):
    # counts generated with IS influence
    counts, patients_labels, _ = preprocess_for_classification(generated_counts)

    # counts generated with similar beta0, but without IS influence
    expected_counts, _, _ = preprocess_for_classification(generated_expected_counts)

    # IS compensation
    t = generated_counts.shape[0]
    filtered_counts, OT_CR_IS, learned_beta = compensation(generated_counts, generated_IS, t, n_CR, distribution, pool, filename)

    columns = ["transcript" + str(num) for num in range(counts.shape[1])]
    y = patients_labels

    # COUNTS
    X = pd.DataFrame(data=counts, columns=columns)
    y = pd.DataFrame(data=y)
    y.head()
    results1 = classify(X, y, n_splits=10, n_repeats=1)
    np.savez('{filename}_origin.npz'.format(filename=filename), X=X, y=y)

    # EXPECTED
    X = pd.DataFrame(data=expected_counts, columns=columns)
    y = pd.DataFrame(data=y)
    y.head()
    results2 = classify(X, y, n_splits=10, n_repeats=1)
    np.savez('{filename}_expected.npz'.format(filename=filename), X=X, y=y)

    # FILTERED
    X = pd.DataFrame(data=filtered_counts, columns=columns)
    y = pd.DataFrame(data=y)
    y.head()
    results3 = classify(X, y, n_splits=10, n_repeats=1)
    np.savez('{filename}_filtered.npz'.format(filename=filename), X=X, y=y)

    return results1, results2, results3

import pandas as pd
import numpy as np
import random
from functions.classificator import classify
from functions.data_preprocess import preprocess_for_classification

def experiment_iteration(STA_range, n_repeats, gen_origin, gen_expected, gen_IS,
                         n_genSTA, n_genCR, n_genOT,
                         distribution, pool, filename):
    n_STA = STA_range[-1]  # maximal number of STA patients

    # arrays for AUC results
    mean_auc_results_origin = np.zeros((n_repeats, len(STA_range)))
    mean_auc_results_expected = np.zeros((n_repeats, len(STA_range)))
    mean_auc_results_filtered = np.zeros((n_repeats, len(STA_range)))

    for j in range(n_repeats):
        STA_random_index = random.sample(range(n_genSTA), n_STA)
        CR_random_index = random.sample(range(n_genCR), 15)
        OT_random_index = random.sample(range(n_genOT), 15)

        counts_STA = gen_origin[0][gen_origin[0].columns[STA_random_index]]
        counts_CR = gen_origin[1][gen_origin[1].columns[CR_random_index]]
        counts_OT = gen_origin[2][gen_origin[2].columns[OT_random_index]]

        expected_STA = gen_expected[0][gen_expected[0].columns[STA_random_index]]
        expected_CR = gen_expected[1][gen_expected[1].columns[CR_random_index]]
        expected_OT = gen_expected[2][gen_expected[2].columns[OT_random_index]]

        IS_STA = gen_IS[0].loc[gen_IS[0].index[STA_random_index].to_numpy()]
        IS_CR = gen_IS[1].loc[gen_IS[1].index[CR_random_index]]
        IS_OT = gen_IS[2].loc[gen_IS[2].index[OT_random_index]]

        a = pd.concat([counts_STA, counts_CR, counts_OT], axis=1)
        b = pd.concat([expected_STA, expected_CR, expected_OT], axis=1)
        c = pd.concat([IS_STA, IS_CR, IS_OT], axis=0)

        patient_labels = a.columns

        for i in range(len(STA_range)):
            # leave only STA_range[i] STA patients
            a_tmp = a.drop(patient_labels[range(n_STA - STA_range[i])], axis=1)
            b_tmp = b.drop(patient_labels[range(n_STA - STA_range[i])], axis=1)
            c_tmp = c.drop(patient_labels[range(n_STA - STA_range[i])], axis=0)
            try:
                d, e, f = experiment(a_tmp, b_tmp, c_tmp, n_CR=15, distribution=distribution, pool=pool, filename=filename)
                mean_auc_results_origin[j, i] = d.mean()
                mean_auc_results_expected[j, i] = e.mean()
                mean_auc_results_filtered[j, i] = f.mean()
            except Exception as e:  #! when STA number is too small, there are the PerfectSeparation error
                print('Exception when ', STA_range[i], ' STA samples. Error: ', e)
                mean_auc_results_origin[j, i] = np.nan
                mean_auc_results_expected[j, i] = np.nan
                mean_auc_results_filtered[j, i] = np.nan
    return mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered

In [17]:
def experiment_classification(n_transcripts, n_IS_effect, IS_effect, n_class_effect, class_effect_mean, STA_range, pool, filename):
    n_repeats = 10
    n_genSTA = 7000 # pocet nagenerovanych STA
    n_genCR = 100
    n_genOT = 100
    _, _, _, gen_origin, gen_expected, gen_IS = rna_seq_generator(
        n_transcripts=n_transcripts,
        distribution=Distribution.POISSON, #"Negative binomial",
        n_STA=n_genSTA, n_CR=n_genCR, n_OT=n_genOT,
        n_IS_effect=n_IS_effect, IS_effect=(IS_effect, IS_effect+0.01),  # IS effect
        is_class_effect=n_class_effect > 0, n_class_effect=n_class_effect, class_effect=NormalDistributionParameters(class_effect_mean, 0.1)
    )
    mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered = \
        experiment_iteration(STA_range, n_repeats,
                             gen_origin, gen_expected, gen_IS,
                             n_genSTA, n_genCR, n_genOT, "Poisson", pool, filename) #"Negative binomial")
    return mean_auc_results_origin, mean_auc_results_expected, mean_auc_results_filtered

In [ ]:
%%time
#
# transcripts = [10, 100, 1000, 10000]
# class_effect_n_transcripts = [0, 2, 5, 10]
# class_effect = [0.2, 0.3, 0.4]  # mean of normal distribution
#
# IS_effect_n_transcripts = [2]#, 5, 10]
# IS_effect = [0.3]#, 0.5, 0.9]  # coefficients beta1..beta3

transcripts = [10000]
class_effect_n_transcripts = [0, 5, 10, 20]
class_effect = [0.3, 0.4]  # mean of normal distribution

IS_effect_n_transcripts = [5, 10]
IS_effect = [0.5, 0.9]  # coefficients beta1..beta3
with Pool() as pool:
    for t in transcripts:
        for n_IS in IS_effect_n_transcripts:
            print('new IS_effect_n_transcripts: ', n_IS)
            for i in IS_effect:
                print('new IS_effect: ', i)
                for n_class in class_effect_n_transcripts:
                    if n_class == 0:
                        c = 0.2
                        filename = 'generateddata06_08/{t}_{n_IS}_{i}_{n_class}_poisson'.format(t=t, n_IS=n_IS, i=i, n_class=n_class)
                        mean_o, mean_e, mean_f = experiment_classification(t, n_IS, i, n_class, c, STA_range, pool, filename)
                        saved_origin[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_origin[t][n_IS][i][n_class][c]['auc'] = mean_o

                        saved_expected[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_expected[t][n_IS][i][n_class][c]['auc'] = mean_e

                        saved_filtered[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                        saved_filtered[t][n_IS][i][n_class][c]['auc'] = mean_f
                    else:
                        for c in class_effect:
                            STA_range = [20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
                            
                            filename = 'generateddata06_08/{t}_{n_IS}_{i}_{n_class}_{c}_poisson'.format(t=t, n_IS=n_IS, i=i, n_class=n_class, c=c)
                            if os.path.isfile(filename + '_origin.npz') or \
                                    os.path.isfile(filename + '_expected.npz') or \
                                    os.path.isfile(filename + '_filtered.npz') or \
                                    os.path.isfile(filename + '_learnedbeta.npz'):
                                print('Warning: Dataset with filename \"' + filename + '\" already exists')
                                continue

                            print('a')
                            mean_o, mean_e, mean_f = experiment_classification(t, n_IS, i, n_class, c, STA_range, pool, filename)
                            saved_origin[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_origin[t][n_IS][i][n_class][c]['auc'] = mean_o

                            saved_expected[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_expected[t][n_IS][i][n_class][c]['auc'] = mean_e

                            saved_filtered[t][n_IS][i][n_class][c]['STA_range'] = STA_range
                            saved_filtered[t][n_IS][i][n_class][c]['auc'] = mean_f

                np.savez('auc_results_origin.npz', auc_data=saved_origin)
                np.savez('auc_results_expected.npz', auc_data=saved_expected)
                np.savez('auc_results_filtered.npz', auc_data=saved_filtered)

new IS_effect_n_transcripts:  2
new IS_effect:  0.3
a


In [ ]:
# Save data
np.savez('auc_results_origin5-poisson.npz', auc_data=saved_origin)
np.savez('auc_results_expected5-poisson.npz', auc_data=saved_expected)
np.savez('auc_results_filtered5-poisson.npz', auc_data=saved_filtered)

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

In [20]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

[1, 4, 9]
